# Carga de Datos

##### Futuras mejoras
* Agregar la diferencia de tiempo en que se consiguió controlar el incendio
* Analizar mejor de qué forma se distribuyen los datos nulos y de qué forma puedo rellenar los valores faltantes, si es posible.

## Importación de librerías

In [ ]:
from os import path
import pandas as pd
import sqlite3
import datetime as dt

## Constantes

Rutas usuales que se ocuparán en el notebook

In [ ]:
# Salvar DataFrame?
SAVE_DF = True

# RUTAS
MAIN_PATH = path.join("..")

DATA_PATH = path.join(MAIN_PATH, "data")
WF_FOLDER_PATH = path.join(DATA_PATH, "wildfires_us")
WF_DATA_PATH = path.join(WF_FOLDER_PATH, "FPA_FOD_20170508.sqlite")
WF_DATA_COLUMNS_PATH = path.join(WF_FOLDER_PATH, "COLUMNS")

## Funciones auxiliares

Función que se ocupará en la carga de datos.

In [ ]:
def crear_puntero(path=WF_DATA_PATH, msg=False):
    """Función que crea un puntero con el dataset de incendios forestales en US.
    """
    try:
        conn = sqlite3.connect(path)
        if msg:
            print("Conexión realizada con éxito.")
    except:
        if msg:
            print("No se encuentra el archivo.")
    return conn

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [ ]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [ ]:
# Todas las columnas
columnas = list(pd.read_csv(WF_DATA_COLUMNS_PATH))

# Columnas que se ocuparán en el análisis
columnas_ocupadas = [
    "FIRE_YEAR",        # Año en que el incendio fue descubierto o confirmado
    "DISCOVERY_DATE",   # Fecha en que fue descubiertoel incendio
    "DISCOVERY_TIME",   # Hora en que fue descubierto el incendio
    "STAT_CAUSE_DESCR", # Descripción de la causa del incendio
    "CONT_DATE",        # Fecha en que se contuvo el incendio
    "CONT_TIME",        # Hora en que se contuvo el incendio
    "FIRE_SIZE",        # Área final estimada que alcanzó el incendio
    "FIRE_SIZE_CLASS",
    "LATITUDE",         # Latitud de la localización del incendio
    "LONGITUDE",        # Longitud de la localización del incendio
    "STATE",
]

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [ ]:
# str que se ocupará al cargar los datos
columnas_str = ",".join(columnas_ocupadas)

# Obtener una fila del dataset
conn = crear_puntero(msg=True)
df = pd.read_sql_query(f"SELECT {columnas_str} FROM 'Fires'", conn)
conn.close()

del columnas_str, conn

print_cantidad(df)

# df = df[df["FIRE_YEAR"].map(int) >= 2011].reset_index()

print_cantidad(df)

df.head()

## Descripción preliminar de los datos

Observemos la información actual del df

In [ ]:
df.info(verbose=True)

Observamos que 6 variables son del tipo numérico, y 5 son strings. Sin embargo, `DISCOVERY_DATE` y `CONT_DATE` son fechas. Esto se tratará más adelante.

### Valores duplicados

Revisamos si no tiene datos duplicados

In [ ]:
n_dup = sum(df.duplicated())
print("Existen", n_dup, "valores duplicados")
del n_dup

Dado que existen valores duplicados en el dataset, los dropeamos.

In [ ]:
df.drop_duplicates(inplace=True, ignore_index=True)
df.info(verbose=True)

### Manejo de valores nulos

Primero observamos el número de elementos nulos que tiene el dataset.

In [ ]:
df.isna().sum()

Dado al gran tamaño de la base de datos, nos podemos dar "el lujo" de eliminar las filas que contengan elementos nulos.

In [ ]:
# Se quitan los valores nulos
df.dropna(inplace=True)

# Se resetean los índices
df.reset_index(drop=True, inplace=True)

# Se imprime la cantidad
print_cantidad(df)

# Y se muestran la cantidad de nulos que tiene actualmente
df.isna().sum()

Ahora revisamos el tipo de datos que tiene el Data Frame

In [ ]:
df.info()

Terminamos con un dataset con la mitad de datos de cómo fue originalmente.

### Manejo de las variables de tiempo

#### Creación de la columnas `{}_DATE_TIME`, a partir de las dos anteriores

Notemos que `{DISCOVERY,CONT}_DATE` está en formato timestamp, y que `{DISCOVERY,CONT}_TIME` está en formato `'hhmm'`:

In [ ]:
cols_aux = ["DISCOVERY_DATE", "DISCOVERY_TIME", 
            "CONT_DATE", "CONT_TIME"]

df[cols_aux].head()

Generamos una nueva columna que tenga la fecha y hora contenida. Pues la fecha está en formato Timestamp y la hora está en formato `'hhmm'`. Lo hacemos para los prefijos `'DISCOVERY_'` y `'CONT_'`.

In [ ]:
def generar_datetime(fecha_col, hora_col):
    """Recibe un DataFrame 'df', una columna 'fecha_col' que tiene 
    la fecha en formato de marca de tiempo (Timestamp) y una columna 
    'hora_col' que tiene la hora en formato 'hhmm'.

    Retorna una columna del tipo 'Series', de datos del tipo datetime,
    con la fecha y hora juntas.
    """

    # Transformamos la columna de fechas a la fecha "usual"
    fecha_col_ = pd.to_datetime(fecha_col - pd.Timestamp(0).to_julian_date(),
                                unit='D',
                                dayfirst=True)
    
    # Obtenemos solo la parte de la fecha como una columna de str
    fecha_col_ = fecha_col_.map(lambda x: str(x).split()[0])

    # Procesamos la columna de la hora para que esté en formato "hh:mm:ss"
    hora_col_ = hora_col.map(lambda x: ":".join([" " + x[0:2], x[2:4], "00"]))

    # Retornamos la suma de ambas, en formato de datetime
    return pd.to_datetime(fecha_col_ + hora_col_)

In [ ]:
# Creamos nuevas columnas
df["DISC_DATE_TIME"] = generar_datetime(df["DISCOVERY_DATE"],
                                        df["DISCOVERY_TIME"])
df["CONT_DATE_TIME"] = generar_datetime(df["CONT_DATE"],
                                        df["CONT_TIME"])

# Agregamos estas nuevas columnas al conjunto de columnas
nuevas_cols = ["DISC_DATE_TIME", "CONT_DATE_TIME"]
columnas += nuevas_cols
columnas_ocupadas += nuevas_cols

df[nuevas_cols].head()

Quitamos ahora las columnas que resultan redundantes, al tener la misma información de los que tiene prefijo `_DATE_TIME`: 
* `DISCOVERY_DATE`, `DISCOVERY_TIME`.
* `CONT_DATE`, `CONT_TIME`.

In [ ]:
for x in cols_aux:
    columnas_ocupadas.remove(x)
    columnas_sin_ocupar.append(x)

del cols_aux, nuevas_cols
    
df = df[columnas_ocupadas]
print_cantidad(df)
df.head()

#### Creación de las columnas `{}_MONTH` y `{}_DAY_OF_WEEK`, a partir de `{}_DATE_TIME`

Crearemos una columna de los meses y semanas, para ver si nos entrega información extra.

In [ ]:
def number_to_month(n):
    """Hace un mapeo del número de un mes, al nombre del mes.
    Retorna None si no se entrega un número entero entre el 1 y el 12.
    """
    
    m = {
        1:"January", 2:"February", 3:"March", 4:"April",
        5:"May", 6:"June", 7:"July", 8:"August",
        9:"September", 10:"Octuber", 11:"November", 12:"December"
    }
    
    return m.get(n, None)

In [ ]:
# Creamos las columnas {DISC,CONT}_{MONTH,DAY_OF_WEEK}
for x in ["DISC", "CONT"]:
    df[f"{x}_MONTH"] = df[f"{x}_DATE_TIME"].dt.month.map(number_to_month)
    df[f"{x}_DOW"] = df[f"{x}_DATE_TIME"].dt.day_name()
    df[f"{x}_TIME"] = df[f"{x}_DATE_TIME"].dt.hour.map(int)

# Añadimos las nuevas columnas al conjunto de columnas
cdt = columnas_ocupadas.pop(-1)
columnas.pop(-1)
cols_aux = ["DISC_MONTH", "DISC_DOW", "DISC_TIME",
            cdt, "CONT_MONTH", "CONT_DOW", "CONT_TIME"]

columnas.extend(cols_aux)
columnas_ocupadas.extend(cols_aux)
del cdt, cols_aux

# Actualizamos el df
df = df[columnas_ocupadas]

df.head()

In [ ]:
df.info()

#### Obtener las diferencias de tiempo del control del incendio

In [ ]:
df["DT_FIRE"] = df["CONT_DATE_TIME"] - df["DISC_DATE_TIME"]
df.head()

In [ ]:
df["DT_FIRE"] = df.DT_FIRE.map(lambda x: x.total_seconds()/3600)
df.head()

## Orden por fecha de descubrimiento

In [ ]:
df.sort_values(by=["DISC_DATE_TIME"], ignore_index=True, inplace=True)

In [ ]:
df.head()

## Salvar DataFrame

Salvamos el DataFrame en la carpeta respectiva.

In [ ]:
if SAVE_DF:
    df.to_csv(path.join(WF_FOLDER_PATH, "WILDFIRES_USA.csv"), index=False)
    print("Dataset guardado.")
    s = ",".join(list(df.columns))
    pd.DataFrame(data={s:[0]}).to_csv(path.join(WF_FOLDER_PATH, 
                                                "WILDFIRES_USA_COLUMNS.csv"), 
                                      index=False)
    print("Columnas del dataset guardadas.")
    del s